In [0]:
!pip install --no-warn-conflicts -q deepctr==0.7.5

In [5]:
!pip install tensorflow==1.4.0

In [6]:
from google.colab import (files,drive)
import os
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
#from tensorflow.keras.models import Model, load_model
from deepctr.models import DIN, DeepFM
from deepctr.inputs import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
#from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
#from tensorflow.keras.utils import get_custom_objects
#from tensorflow.keras.optimizers import Adam,RMSprop
#from tensorflow.keras.layers import Activation
#from tensorflow.keras import backend as K
#from tensorflow.keras import callbacks
#from tensorflow.keras import utils
#import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np
import warnings
import pandas_profiling 
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
warnings.simplefilter('ignore')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or 

In [7]:
drive.mount('/content/gdrive', force_remount=True)
path = "/content/gdrive/"
os.chdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#The original data consists of four tables with raw_sample, ad_features, user_profiles and user_behaviors. We joined the all tables on big query and selected part of data for samples
data = pd.read_csv('My Drive/BAX423+453 Five Guys/BAX423 BIG DATA/Final Project/Data/merge_data_5k/merge_data_5k.csv')

In [9]:
data.head()

,user,btag,cate,brand,num,adgroup_id,userid,cms_segid,cms_group_id,final_gender_code,age_level,pvalue_level,shopping_level,occupation,new_user_class_level_,ad_time_stamp,pid,nonclk,clk,ad_cate_id,campaign_id,customer,ad_brand,price
0,520610,pv,7889,145102,1,77592,520610,0,12,1,6,NaN,3,0,NaN,1494235429,430539_1007,1,0,2249,129207,47213,335936.0,8986.0
1,520610,pv,11702,456398,1,250775,520610,0,12,1,6,NaN,3,0,NaN,1494579781,430539_1007,1,0,11702,244808,217150,NaN,4.2
2,951784,pv,6813,433111,5,138391,951784,0,12,1,6,NaN,1,0,NaN,1494491639,430548_1007,1,0,6813,219977,203097,NaN,61.0
3,739066,pv,6407,33424,1,671288,739066,96,12,1,6,3.0,3,0,NaN,1494336453,430539_1007,1,0,4526,24160,93536,73435.0,799.0
4,739066,pv,4526,327646,1,143461,739066,96,12,1,6,3.0,3,0,NaN,1494336584,430539_1007,1,0,4282,174738,26288,63989.0,358.0


In [10]:
data.count()

user                     2762608
btag                     2762608
cate                     2762608
brand                    2762608
num                      2762608
adgroup_id               2762608
userid                   2762608
cms_segid                2762608
cms_group_id             2762608
final_gender_code        2762608
age_level                2762608
pvalue_level             1249312
shopping_level           2762608
occupation               2762608
new_user_class_level_    2166852
ad_time_stamp            2762608
pid                      2762608
nonclk                   2762608
clk                      2762608
ad_cate_id               2762608
campaign_id              2762608
customer                 2762608
ad_brand                 1938548
price                    2762608
dtype: int64

In [11]:
# drop overlaps with 'user' and useless variables
final_df = data.drop(columns=['userid','cms_segid', 'cms_group_id', 'customer', 'pvalue_level','new_user_class_level_', 'nonclk','pid','ad_time_stamp'])
final_df.head()

,user,btag,cate,brand,num,adgroup_id,final_gender_code,age_level,shopping_level,occupation,clk,ad_cate_id,campaign_id,ad_brand,price
0,520610,pv,7889,145102,1,77592,1,6,3,0,0,2249,129207,335936.0,8986.0
1,520610,pv,11702,456398,1,250775,1,6,3,0,0,11702,244808,NaN,4.2
2,951784,pv,6813,433111,5,138391,1,6,1,0,0,6813,219977,NaN,61.0
3,739066,pv,6407,33424,1,671288,1,6,3,0,0,4526,24160,73435.0,799.0
4,739066,pv,4526,327646,1,143461,1,6,3,0,0,4282,174738,63989.0,358.0


In [12]:
final_df.describe()

,user,cate,brand,num,adgroup_id,final_gender_code,age_level,shopping_level,occupation,clk,ad_cate_id,campaign_id,ad_brand,price
count,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,2.762608e+06,1.938548e+06,2.762608e+06
mean,5.218145e+05,5.653908e+03,2.224584e+05,3.386945e+00,5.155424e+05,1.820269e+00,3.322075e+00,2.863031e+00,4.455319e-02,5.797384e-02,5.372834e+03,2.093112e+05,2.231318e+05,4.984280e+02
std,3.295863e+05,2.448490e+03,1.325319e+05,8.742217e+00,2.181907e+05,3.839629e-01,1.190995e+00,4.250758e-01,2.063207e-01,2.336940e-01,2.304382e+03,1.242508e+05,1.309648e+05,6.387926e+03
min,3.690000e+02,1.000000e+00,2.000000e+00,1.000000e+00,2.300000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,7.000000e+00,3.000000e+00,3.000000e-02
25%,2.296610e+05,4.292000e+03,1.020870e+05,1.000000e+00,3.664560e+05,2.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,4.282000e+03,1.049900e+05,1.035560e+05,8.500000e+01
50%,5.076360e+05,6.177000e+03,2.249850e+05,1.000000e+00,5.680950e+05,2.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,5.920000e+03,2.032880e+05,2.220360e+05,1.680000e+02
75%,7.797190e+05,6.468000e+03,3.367600e+05,3.000000e+00,6.884330e+05,2.000000e+00,4.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,6.426000e+03,3.202440e+05,3.374640e+05,3.250000e+02
max,1.141691e+06,1.297300e+04,4.614030e+05,1.125000e+03,8.467570e+05,2.000000e+00,6.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,1.296000e+04,4.234350e+05,4.612910e+05,9.999990e+05


### Deep Interest Network (DIN)
DIN represents users’ diverse interests with an interest distribution and designs an attention-like network structure to locally activate the related interests according to the candidate ad. Behaviors with higher relevance to the candidate ad get higher attention scores and dominate the prediction.  

In [0]:
final_df['hist_ad_cate_id'] = final_df['cate']
final_df['hist_ad_brand'] = final_df['brand']
final_df['btag'] = final_df['btag'].astype('category').cat.codes

In [0]:
#Construct the sparse,dense and sequence features required for DIN model
sparse_features_1 = ['user', 'final_gender_code','ad_cate_id','ad_brand', 'age_level','shopping_level','btag','occupation']
dense_features_1 = ['price','num']
sequence_features_1 = ['hist_ad_cate_id','hist_ad_brand']

In [15]:
behavior_feature_list = ['ad_cate_id','ad_brand']
behavior_feature_list

['ad_cate_id', 'ad_brand']

In [0]:
final_df[sparse_features_1] = final_df[sparse_features_1].fillna('0', )
final_df[sequence_features_1] = final_df[sequence_features_1].fillna('0', )
final_df[dense_features_1] = final_df[dense_features_1].fillna(0, )
target = ['clk']

In [0]:
#Perform simple transformation on dense features
mms = MinMaxScaler(feature_range=(0, 1))
final_df[dense_features_1] = mms.fit_transform(final_df[dense_features_1])

In [0]:
# SparseFeat('age_level',vocabulary_size=final_df['age_level'].nunique(), embedding_dim = 4),
# SparseFeat('shopping_level',vocabulary_size=final_df['shopping_level'].nunique(), embedding_dim = 4),

In [0]:
feature_columns = [SparseFeat('user',vocabulary_size=1200000, embedding_dim = 10),
                   SparseFeat('adgroup_id',vocabulary_size=1200000, embedding_dim = 4),
                   SparseFeat('ad_cate_id',vocabulary_size=1200000, embedding_dim = 8),
                   SparseFeat('ad_brand',vocabulary_size=1200000, embedding_dim = 8),
                   SparseFeat('final_gender_code',vocabulary_size=1200000, embedding_dim = 4),
                   SparseFeat('age_level',vocabulary_size=1200000, embedding_dim = 4),
                   SparseFeat('shopping_level',vocabulary_size=1200000, embedding_dim = 4),
                   SparseFeat('occupation',vocabulary_size=1200000, embedding_dim = 4),
                   SparseFeat('btag',vocabulary_size=1200000, embedding_dim = 4),
                   DenseFeat('price',1),
                   DenseFeat('num',1)]

In [0]:
feature_columns += [VarLenSparseFeat(SparseFeat('hist_ad_cate_id', vocabulary_size=1200000,embedding_dim=8, embedding_name = 'ad_cate_id'), maxlen=1),
                    VarLenSparseFeat(SparseFeat('hist_ad_brand', vocabulary_size=1200000,embedding_dim=8, embedding_name = 'ad_brand'), maxlen=1)]

In [21]:
feature_columns

[SparseFeat(name='user', vocabulary_size=1200000, embedding_dim=10, use_hash=False, dtype='int32', embedding_name='user', group_name='default_group'),
 SparseFeat(name='adgroup_id', vocabulary_size=1200000, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='adgroup_id', group_name='default_group'),
 SparseFeat(name='ad_cate_id', vocabulary_size=1200000, embedding_dim=8, use_hash=False, dtype='int32', embedding_name='ad_cate_id', group_name='default_group'),
 SparseFeat(name='ad_brand', vocabulary_size=1200000, embedding_dim=8, use_hash=False, dtype='int32', embedding_name='ad_brand', group_name='default_group'),
 SparseFeat(name='final_gender_code', vocabulary_size=1200000, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='final_gender_code', group_name='default_group'),
 SparseFeat(name='age_level', vocabulary_size=1200000, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='age_level', group_name='default_group'),
 SparseFeat(name='shopping_leve

In [0]:
feature_names = get_feature_names(feature_columns)

In [0]:
train, test = train_test_split(final_df, test_size=0.2)
train_model_input = {name:train[name] for name in feature_names}
test_model_input = {name:test[name] for name in feature_names}

In [0]:
model = DIN(feature_columns, behavior_feature_list, dnn_use_bn=True,
        dnn_hidden_units=(200, 80), dnn_activation='relu', att_hidden_size=(80, 40), att_activation="dice",
        att_weight_normalization=False, l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, init_std=0.0001, seed=1024,
        task='binary')

In [25]:
model.input

[<tf.Tensor 'user_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'adgroup_id_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'ad_cate_id_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'ad_brand_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'final_gender_code_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'age_level_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'shopping_level_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'occupation_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'btag_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'price_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'num_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'hist_ad_cate_id_1:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'hist_ad_brand_1:0' shape=(?, 1) dtype=int32>]

In [26]:
#Define Model,Train,Predict and Evaluate
model.compile("adam", "binary_crossentropy",metrics=['binary_crossentropy'], )
history = model.fit(train_model_input, train[target].values,batch_size=5024, epochs=5, verbose=1, validation_split=0.25, )
pred_ans = model.predict(test_model_input, batch_size=256)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 2))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 2))

Train on 1657564 samples, validate on 552522 samples
Epoch 1/5
1657564/1657564 [==============================] - 485s - loss: 0.1402 - binary_crossentropy: 0.1401 - val_loss: 0.6405 - val_binary_crossentropy: 0.6404
Epoch 2/5
1657564/1657564 [==============================] - 474s - loss: 0.0442 - binary_crossentropy: 0.0440 - val_loss: 0.6116 - val_binary_crossentropy: 0.6114
Epoch 3/5
1657564/1657564 [==============================] - 472s - loss: 0.0350 - binary_crossentropy: 0.0348 - val_loss: 0.5904 - val_binary_crossentropy: 0.5901
Epoch 4/5
1657564/1657564 [==============================] - 476s - loss: 0.0315 - binary_crossentropy: 0.0312 - val_loss: 0.5720 - val_binary_crossentropy: 0.5717
Epoch 5/5
1657564/1657564 [==============================] - 482s - loss: 0.0296 - binary_crossentropy: 0.0292 - val_loss: 0.5725 - val_binary_crossentropy: 0.5720
test LogLoss 0.57
test AUC 0.78
